In [2]:
import os

os.environ["TAVILY_API_KEY"] = ''


In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("Узнай на weatherapi погоду в Москве")
print(search_results)
# Если нужно, вы можете создать другие инструменты.
# После создания всех необходимых инструментов
# их можно сохранить в списке, к которому можно обращаться позднее.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Москва', 'region': 'Moscow City', 'country': 'Россия', 'lat': 55.75, 'lon': 37.62, 'tz_id': 'Europe/Moscow', 'localtime_epoch': 1725888224, 'localtime': '2024-09-09 16:23'}, 'current': {'last_updated_epoch': 1725887700, 'last_updated': '2024-09-09 16:15', 'temp_c': 25.4, 'temp_f': 77.7, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 110, 'wind_dir': 'ESE', 'pressure_mb': 1022.0, 'pressure_in': 30.18, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 39, 'cloud': 0, 'feelslike_c': 25.2, 'feelslike_f': 77.3, 'windchill_c': 26.0, 'windchill_f': 78.8, 'heatindex_c': 25.6, 'heatindex_f': 78.0, 'dewpoint_c': 8.3, 'dewpoint_f': 46.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 7.0, 'gust_mph': 13.9, 'gust_kph': 22.3}}"}, {'url': 'https://weather.rambler.ru/v-moskve/9-september/', 'content': 'Подробный прогн

In [4]:
from langchain.chat_models.gigachat import GigaChat

model = GigaChat(
    credentials="",
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
)

In [5]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="Привет!")])
response.content

'Здравствуйте! Я могу вам чем-нибудь помочь?'

In [6]:
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="Привет!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Здравствуйте! Я могу вам чем-нибудь помочь?
ToolCalls: []


In [7]:
response = model_with_tools.invoke([HumanMessage(content="Какая погода в Москве, спроси у weatherapi?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Какая погода в Москве, спроси у weatherapi?'}, 'id': '787ade41-7d17-48de-a190-0670b0cc4b74', 'type': 'tool_call'}]


In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

response = agent_executor.invoke({"messages": [HumanMessage(content="Привет!")]})

response["messages"]

[HumanMessage(content='Привет!', id='d4f663fc-4f7c-40c8-b8c9-6dcd074438f0'),
 AIMessage(content='Здравствуйте! Я могу вам чем-нибудь помочь?', response_metadata={'token_usage': Usage(prompt_tokens=86, completion_tokens=16, total_tokens=102), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'stop'}, id='run-b4d65e21-ceb2-418a-9859-0cee2f6e36b2-0')]

In [9]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Какая погода в Москве, узнай на weatherapi?")]}
)
response["messages"]

[HumanMessage(content='Какая погода в Москве, узнай на weatherapi?', id='253f426f-8f0b-40dc-9c2f-28cd6cd31434'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'какая погода в москве, узнать на weatherapi?'}}}, response_metadata={'token_usage': Usage(prompt_tokens=96, completion_tokens=31, total_tokens=127), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'function_call'}, id='run-9897634a-cc7f-44ab-92ad-5b28c44e2c7c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'какая погода в москве, узнать на weatherapi?'}, 'id': 'a3ea1703-d695-462f-b808-753cab38aac0', 'type': 'tool_call'}]),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'\\u041c\\u043e\\u0441\\u043a\\u0432\\u0430\', \'region\': \'Moscow City\', \'country\': \'\\u0420\\u043e\\u0441\\u0441\\u0438\\u044f\', \'lat\': 55.75, \'lon\': 37.62, \'tz_id\': \'Europe/Moscow\', \'loc

In [10]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Какая погода в Москве, узнай на weatherapi?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'какая погода в москве, узнать на weatherapi?'}}}, response_metadata={'token_usage': Usage(prompt_tokens=96, completion_tokens=31, total_tokens=127), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'function_call'}, id='run-1587b101-3803-43c0-b364-a1f2ba7db3e1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'какая погода в москве, узнать на weatherapi?'}, 'id': 'f0c49deb-d20e-4c0e-ab80-0655e5f0aed6', 'type': 'tool_call'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Moscow\', \'region\': \'Moscow City\', \'country\': \'Russia\', \'lat\': 55.75, \'lon\': 37.62, \'tz_id\': \'Europe/Moscow\', \'localtime_epoch\': 1725888711, \'localtime\': \'2024-09-09 16:31\'}, \'current\': {\'last_updated_epoch\': 1725888600, \'last_updated\'

In [14]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}


In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Привет! Меня зовут Вася")]}, config
) :
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Здравствуйте, Вася! Рада слышать вас.', response_metadata={'token_usage': Usage(prompt_tokens=92, completion_tokens=17, total_tokens=109), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'stop'}, id='run-6903a813-1f4f-4d17-b855-0e1318eeec6c-0')]}}
----


In [16]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Как меня зовут?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Вас зовут Вася.', response_metadata={'token_usage': Usage(prompt_tokens=121, completion_tokens=9, total_tokens=130), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'stop'}, id='run-e0a56819-5602-44c0-bc88-b8a4cf2bada2-0')]}}
----


In [18]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Какая погода в омске?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'какая погода в омске'}}}, response_metadata={'token_usage': Usage(prompt_tokens=136, completion_tokens=25, total_tokens=161), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'function_call'}, id='run-d714f665-ed86-49d8-b921-39b92b516b9e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'какая погода в омске'}, 'id': '082c14f2-e133-4e9b-b435-d39ccbbe266a', 'type': 'tool_call'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://pogoda.mail.ru/prognoz/omsk/14dney/", "content": "12:21. \\u0424\\u0430\\u0437\\u0430 \\u041b\\u0443\\u043d\\u044b. \\u043f\\u043e\\u043b\\u043d\\u043e\\u043b\\u0443\\u043d\\u0438\\u0435. \\u041f\\u043e\\u0434\\u0440\\u043e\\u0431\\u043d\\u043e \\u043e \\u043f\\u043e\\u0433\\u043e\\u0434\\u0435 \\u043d\\u0430 14 \\u0434\\u043d\\u0435\\u0439 \\u0432 \\u041e\\u043c\\u0441